# Training Materials: Dam Water Level Change Detection with ALOS-2

**Overall Goal:** To equip participants with the skills to detect changes in dam water levels using ALOS-2 satellite data.

## Module 1: Introduction to ALOS-2 for Water Monitoring

* **1.1 Introduction to ALOS-2 (Daichi-2)**
    * Overview, mission objectives, sensor capabilities (SAR), advantages of L-band SAR.
    * Reference: 8.1.1 a, b
* **1.2 ALOS-2 Data Applications**
    * Focus on disaster monitoring and hydrology/water resource management.
    * Reference: 8.1.1 c
* **1.3 Why use ALOS-2 for Dam Monitoring?** (Highlighting the benefits of SAR for water detection)
* **1.4 Case Study: Yamba Dam, Japan** (Introduce the case study area and the specific problem)

## Module 2: Working with ALOS-2 Data in Python

* **2.1 Data Downloading**
    * Downloading ALOS-2 data from OwnCloud (or other sources).
* **2.2 Reading and Visualizing ALOS-2 Data with Rasterio and Matplotlib**
    * Loading ALOS-2 data using Rasterio.
    * Visualizing ALOS-2 data using Matplotlib (including colormaps).
    * Computing and visualizing histograms.
* **2.2 Data Preprocessing**
    * Speckle filtering: Speckle noise, a granular interference pattern, is inherent in SAR imagery. Speckle filtering reduces this noise, improving image clarity.
    * Radiometric calibration (converting digital numbers to backscatter).
    * Visualizing calibrated data.
    * Computing and visualizing histograms.

In [ ]:
%cd /home/jovyan/shared/Arissara/ALOS-2/Yamba/Finalize

!wget --content-disposition "https://owncloud.glodal-inc.net/owncloud/index.php/s/fORunC9aDo38G5s/download"
!unzip "Yamba.zip"
!pip install rasterio matplotlib

import rasterio
import matplotlib.pyplot as plt

alos_file = '/home/jovyan/shared/Arissara/ALOS-2/Yamba/Finalize/Yamba/IMG-HH-ALOS2310942880-200225-UBSR2.1GUD.tif'

with rasterio.open(alos_file) as src:
    alos_data = src.read(1) 
    transform = src.transform
    crs = src.crs

print(f"CRS: {crs}")
print(f"Transform: {transform}")
print(f"ALOS Data Shape: {alos_data.shape}")

plt.figure(figsize=(10, 10))
plt.imshow(alos_data, cmap='gray')
plt.title('ALOS Data (Band 1)')
plt.colorbar(label='Pixel values')
plt.show()

plt.figure(figsize=(10, 10))
plt.imshow(alos_data, cmap='turbo')
plt.title('ALOS Data (Band 1) with Viridis Colormap')
plt.colorbar(label='Pixel values')
plt.show()

import numpy as np

hist_values, bin_edges = np.histogram(alos_data, bins=256, range=(np.min(alos_data), np.max(alos_data)))

plt.figure(figsize=(8, 6))
plt.plot(bin_edges[:-1], hist_values, lw=2)
plt.title('Histogram of ALOS Data (Band 1)')
plt.xlabel('Pixel values')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

## Module 3: Water Classification with Otsu's Method

* **3.1 Principles of SAR Backscatter for Water Detection**
    * How SAR interacts with water and non-water surfaces.
    * Interpreting backscatter values.
    * Limitations of ALOS-2 data for water detection (e.g., shadows, bare land).
* **3.2 Introduction to Otsu's Method**
    * Theory and concept of Otsu's thresholding for image segmentation.
    * Minimizing intraclass variance.
* **3.3 Implementing Otsu's Method in Python (Scikit-image)**
    * Classifying water using Otsu's method.
    * Visualizing the water mask.
* **3.4 Noise Removal and Post-processing**
    * Applying median filters for noise reduction.
* **3.5 Area Calculation**
    * Calculating the water area from the classified image.
* **3.6 Raster to Vector Conversion (Optional)**
    * Converting the raster water mask to a vector polygon.

In [ ]:
!pip install rasterio geopandas numpy matplotlib scipy scikit-image

In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio.mask
from scipy.ndimage import uniform_filter, median_filter
from skimage import filters, io, color, img_as_float
from skimage.restoration import denoise_tv_chambolle
from skimage.morphology import disk
from skimage.filters import threshold_otsu
from rasterio.features import shapes
from shapely.geometry import shape
from skimage.util import img_as_float

In [ ]:
# --- Load image ---
ALOS_image_path = '/.../IMG-HH-ALOS2294160720-191103-UBSR2.1GUA.tif' # Replace with the path to your input file
with rasterio.open(ALOS_image_path) as src:
    original_image = src.read(1)
    profile = src.profile

In [ ]:
# --- Lee filter ---
def lee_filter(image, window_size=3):
    image = img_as_float(image)
    filtered_image = np.zeros_like(image)
    half_window = window_size // 2
    for i in range(half_window, image.shape[0] - half_window):
        for j in range(half_window, image.shape[1] - half_window):
            window = image[i-half_window:i+half_window+1, j-half_window:j+half_window+1]
            mean = np.mean(window)
            variance = np.var(window)
            local_variance = np.var(window)
            k = local_variance / (local_variance + variance) if local_variance > 0 else 0
            filtered_image[i, j] = mean + k * (image[i, j] - mean)
    return filtered_image

In [ ]:
# Apply Lee filter
filtered_image = lee_filter(original_image, window_size=5)

In [ ]:
# --- Calibration ---
calibrated_image = (10 * np.log10(filtered_image**2)) - 83

In [ ]:
# --- Visualization ---
def plot_image(image, title, cmap='gray', vmin=None, vmax=None):
    plt.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.title(title)
    plt.axis('off')

def plot_histogram(image, title, bins=256):
    plt.hist(image.ravel(), bins=bins, color='blue', alpha=0.7)
    plt.title(title)
    plt.xlabel('Pixel Values')
    plt.ylabel('Frequency')

# Plot images
plt.figure(figsize=(18, 8))
plt.subplot(2, 3, 1)
plot_image(original_image, 'Original Image')

plt.subplot(2, 3, 2)
plot_image(filtered_image, 'Filtered Image')

plt.subplot(2, 3, 3)
plot_image(calibrated_image, 'Calibrated Image')

# Plot histograms
plt.subplot(2, 3, 4)
plot_histogram(original_image, 'Original Image Histogram')

plt.subplot(2, 3, 5)
plot_histogram(filtered_image, 'Filtered Image Histogram')

plt.subplot(2, 3, 6)
plot_histogram(calibrated_image, 'Calibrated Image Histogram')

plt.tight_layout()
plt.show()

In [ ]:
# --- Otsu's and masking ---
threshold = threshold_otsu(filtered_image)
water_mask = calibrated_image < threshold

In [ ]:
# --- Noise removal ---
# Define the size of the median filter (footprint size equivalent to disk radius 2)
water_mask_filtered = median_filter(water_mask, size=3) # Use size 3 for a 3x3 window

# Visualize the calibrated image, water mask, and median-filtered mask
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))

# Display filtered ALOS-2 image
vmin = np.percentile(calibrated_image, 2)
vmax = np.percentile(calibrated_image, 98)
ax1.imshow(calibrated_image, cmap='gray', vmin=vmin, vmax=vmax)
ax1.set_title('Calibrated ALOS-2 Image')
ax1.axis('off')

# Display the original water mask
ax2.imshow(water_mask, cmap='Blues')
ax2.set_title('Original Water Mask from Otsu')
ax2.axis('off')

# Display the median-filtered water mask
ax3.imshow(water_mask_filtered, cmap='Blues')
ax3.set_title('Median Filtered Water Mask')
ax3.axis('off')


plt.show()

In [ ]:
# --- Area calculation ---
pixel_width_m = abs(profile['transform'][0])
pixel_height_m = abs(profile['transform'][4])
pixel_area = pixel_width_m * pixel_height_m
water_area_ha = np.sum(water_mask_filtered) * pixel_area / 10000
print(f'Water Area: {water_area_ha:.2f} hectares')

In [ ]:
# --- Raster to vector (Optional) ---
results = (
    {'properties': {'value': v}, 'geometry': shape(s)}
    for s, v in shapes(filtered_bands.astype(np.int16), transform=profile['transform'])
)
geoms = list(results)
gdf = gpd.GeoDataFrame.from_features(geoms, crs=profile['crs'].to_string())
output_vector = '/home/jovyan/shared/Arissara/ALOS-2/Yamba/Finalize/Output-8.2/rm-noise_water_mask_Yamba.gpkg'
gdf.to_file(output_vector, driver="GPKG")

# --- Visualization (optional) ---
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(calibrated_image, cmap='gray')
plt.title('Calibrated Image')
plt.subplot(1, 2, 2)
plt.imshow(filtered_bands, cmap='Blues')
plt.title('Water Mask')
plt.show()

## Module 4: Change Detection with Time-Series ALOS-2 Data

* **4.1 Introduction to Time-Series Analysis**
    * Basic concepts of time-series data.
* **4.2 Preparing Time-Series Data**
    * Clipping images (Optional), Lee filtering, and calibration data.
    * Layer stacking.
* **4.3 Visualizing Change with QGIS Temporal/Spectral Profile Tool**
    * Visualizing changes in water extent over time.
* **4.4 Interpreting Time-Series Results**
    * Analyzing trends and patterns in water level fluctuations.
    * Relating changes to external factors (e.g., rainfall, dam operations).
* **4.5 (Optional) Accuracy Assessment and Confusion Matrix**
    * Understanding accuracy assessment in remote sensing.
    * Creating and interpreting confusion matrices.

In [ ]:
!pip install numpy matplotlib opencv-python scikit-image rasterio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
from skimage import img_as_float

In [ ]:
# Define folder paths
folder_path = "/home/jovyan/shared/Arissara/ALOS-2/Yamba/Finalize/Yamba"
output_folder = "/home/jovyan/shared/Arissara/ALOS-2/Yamba/Finalize/Output-8.3"
os.makedirs(output_folder, exist_ok=True)

# Gather image file paths
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]

In [ ]:
filtered_images = []
calibrated_images = []

In [ ]:
# Define Lee filter functions and calibration

def lee_filter(image, window_size=3):
    image = img_as_float(image)
    filtered_image = np.zeros_like(image)
    half_window = window_size // 2
    for i in range(half_window, image.shape[0] - half_window):
        for j in range(half_window, image.shape[1] - half_window):
            window = image[i-half_window:i+half_window+1, j-half_window:j+half_window+1]
            mean = np.mean(window)
            variance = np.var(window)
            local_variance = np.var(window)
            k = local_variance / (local_variance + variance) if local_variance > 0 else 0
            filtered_image[i, j] = mean + k * (image[i, j] - mean)
    return filtered_image

def calibrate_image(image):
    return (10 * np.log10(image**2)) - 83

In [ ]:
# Process each image
for file in image_files:
    with rasterio.open(file) as src:
        img = src.read(1)
        profile = src.profile  # Retrieve profile for saving later

    # Apply Lee filter 
    filtered_img = lee_filter(img, window_size=5)
    filtered_images.append(filtered_img)

    # Apply calibration 
    calibrated_img = calibrate_image(filtered_img)
    calibrated_images.append(calibrated_img)

In [ ]:
# Save filtered and calibrated images
for i, filtered_img in enumerate(filtered_images):
    filtered_img_file = os.path.join(output_folder, f"filtered_image_{i+1}.tif")
    profile.update(dtype=rasterio.float32)
    with rasterio.open(filtered_img_file, 'w', **profile) as dst:
        dst.write(filtered_img.astype(np.float32), 1)

for i, calibrated_img in enumerate(calibrated_images):
    calibrated_img_file = os.path.join(output_folder, f"calibrated_image_{i+1}.tif")
    profile.update(dtype=rasterio.float32)
    with rasterio.open(calibrated_img_file, 'w', **profile) as dst:
        dst.write(calibrated_img.astype(np.float32), 1)

print("Processing complete. Images saved to:", output_folder)

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os

# Assuming `calibrated_images` is a list of calibrated 2D arrays
# Stack the images
stacked_image = np.stack(calibrated_images, axis=0)

# Update profile to match stacked image
profile.update(count=stacked_image.shape[0], crs='EPSG:8692')  # Adjust CRS as needed

# Define the output file path for the stacked image
stacked_img_file = os.path.join(output_folder, "stacked_image.tif")

# Save the stacked image
with rasterio.open(stacked_img_file, 'w', **profile) as dst:
    dst.write(stacked_image.astype(np.float32))

print(f"Stacked image saved to: {stacked_img_file}")

# Visualize each band in the stacked image
with rasterio.open(stacked_img_file) as src:
    for band in range(1, src.count + 1):  # `src.count` gives the total number of bands
        img = src.read(band)  # Read the band
        plt.figure(figsize=(8, 8))
        plt.imshow(img, cmap='gray')
        plt.title(f'Band {band}')
        plt.colorbar()
        plt.axis('off')
        plt.show()

In [ ]:
# Stack image
stacked_image = np.stack(calibrated_images, axis=0)
profile.update(count=7, crs='EPSG:8692')  # Or appropriate EPSG
stacked_img_file = os.path.join(output_folder, "stacked_image.tif")
with rasterio.open(stacked_img_file, 'w', **profile) as dst:
    dst.write(stacked_image.astype(np.float32))

with rasterio.open(stacked_img_file) as src:
    for band in range(1, src.count + 1):
        img = src.read(band)
        plt.figure(figsize=(8, 8))
        plt.imshow(img, cmap='gray')
        plt.title(f'Band {band}')
        plt.colorbar()
        plt.show()